In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


def perplexity(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    text: str,
    max_input_length: int = None,
):
    """
    Computes perplexity of a piece of text, measured on a reference model.
    Text is truncated to max_input_length tokens.
    """

    inputs = tok(
        [text], return_tensors="pt", max_length=max_input_length, truncation=True
    ).to("cuda")
    output = model(**inputs).logits
    logits = torch.nn.functional.log_softmax(output, dim=2)
    log_probs = torch.gather(logits[:, :-1, :], 2, inputs["input_ids"][:, 1:, None])[0]

    # Perplexity = exp(-1/N * log P(x_1, ..., x_n))
    return torch.exp(-1 / inputs["input_ids"].size(1) * log_probs.sum()).item()

In [7]:
# gpt2model = AutoModelForCausalLM.from_pretrained("/root/autodl-tmp/gpt2-xl").cuda()
gpt2tok = AutoTokenizer.from_pretrained("/root/autodl-tmp/gpt2-xl")

In [29]:
logits,p,inputs = perplexity(model=gpt2model,tok=gpt2tok,text="Tesla's founder is")
print(p)

19.314800262451172


In [30]:
logits.shape

torch.Size([1, 4, 50257])

In [4]:
import subprocess
import os,sys
os.chdir("/root/EasyEdit")
sys.path.append(os.getcwd())
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams

#新三元组
hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/gpt2-xl.yaml')
prompts = ["Tesla's founder is"]
ground_truth = ['Musk']
target_new = ['Gates']
subject = ['Tesla']
editor=BaseEditor.from_hparams(hparams)
#改变模型参数
metrics, edited_model_false, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)

2024-05-28 15:47:09,664 - easyeditor.editors.editor - INFO - Instantiating model
05/28/2024 15:47:09 - INFO - easyeditor.editors.editor -   Instantiating model
/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
100%|██████████| 1/1 [00:00<00:00,  1.35it/s]
0it [00:00, ?it/s]

MEMIT request sample: [Tesla's founder is] -> [ Gates]
Cached context templates [['{}'], ['The first thing I did when I got here was. {}', 'Therefore, we need to look at the different types. {}', 'Because the two-year contract is only worth £. {}', 'I think we need to look at the fact that. {}', "You'll have to wait for the next update.. {}"]]
Computing right vector (v)
Lookup index found: 0 | Sentence: Tesla's founder is | Token: Tesla
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 12.85 = 12.85 + 0.0 + 0.0 avg prob of [ Gates] 3.5276216294732876e-06
loss 11.454 = 11.452 + 0.002 + 0.0 avg prob of [ Gates] 1.3661512639373541e-05
loss 10.239 = 10.233 + 0.006 + 0.0 avg prob of [ Gates] 5.783430970041081e-05
loss 8.994 = 8.984 + 0.01 + 0.0 avg prob of [ Gates] 0.00026183464797213674
loss 7.557 = 7.541 + 0.016 + 0.0 avg prob of [ Gates] 0.0014073284110054374
loss 5.859 = 5.837 + 0.021 + 0.0 avg prob of [ Gates] 0.009492691606283188
loss 4.038 = 4.

  0%|          | 0/1000 [00:00<?, ?it/s]

orig norm tensor(117.1293, device='cuda:0')
upd norm tensor(2.9498, device='cuda:0', dtype=torch.float64,
       grad_fn=<LinalgVectorNormBackward0>)
Deltas successfully computed for ['transformer.h.17.mlp.c_proj.weight']


2024-05-28 15:47:28,441 - easyeditor.editors.editor - INFO - Execution 0 editing took 5.458102703094482
05/28/2024 15:47:28 - INFO - easyeditor.editors.editor -   Execution 0 editing took 5.458102703094482
2024-05-28 15:47:28,515 - easyeditor.editors.editor - INFO - Evaluation took 0.07196617126464844
05/28/2024 15:47:28 - INFO - easyeditor.editors.editor -   Evaluation took 0.07196617126464844
2024-05-28 15:47:28,516 - easyeditor.editors.editor - INFO - 0 editing: Tesla's founder is -> Gates  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': "Tesla's founder is", 'target_new': 'Gates', 'ground_truth': 'Musk', 'portability': {}, 'locality': {}, 'subject': 'Tesla'}, 'time': 5.458102703094482, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
05/28/2024 15:47:28 - INFO - easyeditor.editors.editor -   0 editing: Tesla's founder is -> Gates  
 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrit

New weights successfully inserted into ['transformer.h.17.mlp.c_proj.weight']


In [9]:
p= perplexity(model=edited_model_false,tok=gpt2tok,text="Tesla's founder is")

In [10]:
print(p)

19.28416633605957
